## README

Start by installing `uv` and make sure you use `uv init mcp_project` to start a new project folder.

```bash
pip install uv
```

```bash
uv init mcp_project
cd mcp_project
```

Now you are in the `mcp_project` folder. You need to add the following dependencies:

```bash
uv add anthropic python-dotenv nest_asyncio
```

And then we install the following dependencies:

```bash
uv add anyio distro httpx jiter pydantic sniffio typing-extensions
```

Next you will to make sure you have dependency `.py` scripts. Save the following as `research_server.py` script in the `mcp_project` directory.

```bash
import arxiv
import json
import os
from typing import List
from mcp.server.fastmcp import FastMCP


PAPER_DIR = "papers"

# Initialize FastMCP server
mcp = FastMCP("research")

@mcp.tool()
def search_papers(topic: str, max_results: int = 5) -> List[str]:
    """
    Search for papers on arXiv based on a topic and store their information.
    
    Args:
        topic: The topic to search for
        max_results: Maximum number of results to retrieve (default: 5)
        
    Returns:
        List of paper IDs found in the search
    """
    
    # Use arxiv to find the papers
    client = arxiv.Client()

    # Search for the most relevant articles matching the queried topic
    search = arxiv.Search(
        query = topic,
        max_results = max_results,
        sort_by = arxiv.SortCriterion.Relevance
    )

    papers = client.results(search)
    
    # Create directory for this topic
    path = os.path.join(PAPER_DIR, topic.lower().replace(" ", "_"))
    os.makedirs(path, exist_ok=True)
    
    file_path = os.path.join(path, "papers_info.json")

    # Try to load existing papers info
    try:
        with open(file_path, "r") as json_file:
            papers_info = json.load(json_file)
    except (FileNotFoundError, json.JSONDecodeError):
        papers_info = {}

    # Process each paper and add to papers_info  
    paper_ids = []
    for paper in papers:
        paper_ids.append(paper.get_short_id())
        paper_info = {
            'title': paper.title,
            'authors': [author.name for author in paper.authors],
            'summary': paper.summary,
            'pdf_url': paper.pdf_url,
            'published': str(paper.published.date())
        }
        papers_info[paper.get_short_id()] = paper_info
    
    # Save updated papers_info to json file
    with open(file_path, "w") as json_file:
        json.dump(papers_info, json_file, indent=2)
    
    print(f"Results are saved in: {file_path}")
    
    return paper_ids

@mcp.tool()
def extract_info(paper_id: str) -> str:
    """
    Search for information about a specific paper across all topic directories.
    
    Args:
        paper_id: The ID of the paper to look for
        
    Returns:
        JSON string with paper information if found, error message if not found
    """

    for item in os.listdir(PAPER_DIR):
        item_path = os.path.join(PAPER_DIR, item)
        if os.path.isdir(item_path):
            file_path = os.path.join(item_path, "papers_info.json")
            if os.path.isfile(file_path):
                try:
                    with open(file_path, "r") as json_file:
                        papers_info = json.load(json_file)
                        if paper_id in papers_info:
                            return json.dumps(papers_info[paper_id], indent=2)
                except (FileNotFoundError, json.JSONDecodeError) as e:
                    print(f"Error reading {file_path}: {str(e)}")
                    continue
    
    return f"There's no saved information related to paper {paper_id}."


if __name__ == "__main__":
    # Initialize and run the server
    mcp.run(transport='stdio')
```

Last step is to make sure we have the main `mcp_chatbot.py` script. You can use the following to save it in the `mcp_project` folder.

```bash
%%writefile mcp_project/mcp_chatbot.py
from dotenv import load_dotenv
from anthropic import Anthropic
from mcp import ClientSession, StdioServerParameters, types
from mcp.client.stdio import stdio_client
from typing import List
import asyncio
import nest_asyncio

nest_asyncio.apply()

load_dotenv()

class MCP_ChatBot:

    def __init__(self):
        # Initialize session and client objects
        self.session: ClientSession = None
        self.anthropic = Anthropic()
        self.available_tools: List[dict] = []

    async def process_query(self, query):
        messages = [{'role':'user', 'content':query}]
        response = self.anthropic.messages.create(max_tokens = 2024,
                                      model = 'claude-3-7-sonnet-20250219',
                                      tools = self.available_tools, # tools exposed to the LLM
                                      messages = messages)
        process_query = True
        while process_query:
            assistant_content = []
            for content in response.content:
                if content.type =='text':
                    print(content.text)
                    assistant_content.append(content)
                    if(len(response.content) == 1):
                        process_query= False
                elif content.type == 'tool_use':
                    assistant_content.append(content)
                    messages.append({'role':'assistant', 'content':assistant_content})
                    tool_id = content.id
                    tool_args = content.input
                    tool_name = content.name
    
                    print(f"Calling tool {tool_name} with args {tool_args}")
                    
                    # Call a tool
                    #result = execute_tool(tool_name, tool_args): not anymore needed
                    # tool invocation through the client session
                    result = await self.session.call_tool(tool_name, arguments=tool_args)
                    messages.append({"role": "user",
                                      "content": [
                                          {
                                              "type": "tool_result",
                                              "tool_use_id":tool_id,
                                              "content": result.content
                                          }
                                      ]
                                    })
                    response = self.anthropic.messages.create(max_tokens = 2024,
                                      model = 'claude-3-7-sonnet-20250219',
                                      tools = self.available_tools,
                                      messages = messages)
                    
                    if(len(response.content) == 1 and response.content[0].type == "text"):
                        print(response.content[0].text)
                        process_query= False

    
    
    async def chat_loop(self):
        """Run an interactive chat loop"""
        print("\nMCP Chatbot Started!")
        print("Type your queries or 'quit' to exit.")
        
        while True:
            try:
                query = input("\nQuery: ").strip()
        
                if query.lower() == 'quit':
                    break
                    
                await self.process_query(query)
                print("\n")
                    
            except Exception as e:
                print(f"\nError: {str(e)}")
    
    async def connect_to_server_and_run(self):
        # Create server parameters for stdio connection
        server_params = StdioServerParameters(
            command="uv",  # Executable
            args=["run", "research_server.py"],  # Optional command line arguments
            env=None,  # Optional environment variables
        )
        async with stdio_client(server_params) as (read, write):
            async with ClientSession(read, write) as session:
                self.session = session
                # Initialize the connection
                await session.initialize()
    
                # List available tools
                response = await session.list_tools()
                
                tools = response.tools
                print("\nConnected to server with tools:", [tool.name for tool in tools])
                
                self.available_tools = [{
                    "name": tool.name,
                    "description": tool.description,
                    "input_schema": tool.inputSchema
                } for tool in response.tools]
    
                await self.chat_loop()


async def main():
    chatbot = MCP_ChatBot()
    await chatbot.connect_to_server_and_run()
  

if __name__ == "__main__":
    asyncio.run(main())
```

In [ ]:
pip install uv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 22.1 MB/s eta 0:00:00


In [ ]:
! uv init mcp_project
! cd mcp_project

Initialized project `mcp-project` at `/content/mcp_project`


In [ ]:
! cd mcp_project/

In [ ]:
import os

os.chdir("mcp_project")

In [ ]:
! ls -l

total 8
-rw-r--r-- 1 root root  89 Jun  2 17:22 main.py
-rw-r--r-- 1 root root 157 Jun  2 17:22 pyproject.toml
-rw-r--r-- 1 root root   0 Jun  2 17:22 README.md


In [ ]:
! uv add anyio distro httpx jiter pydantic sniffio typing-extensions

Using CPython 3.11.12 interpreter at: /usr/bin/python3
Creating virtual environment at: .venv
Resolved 15 packages in 396ms
⠙ Preparing packages... (0/14)
⠙ Preparing packages... (0/14)
⠙ Preparing packages... (0/14)
annotated-types      ------------------------------     0 B/13.32 KiB
⠙ Preparing packages... (0/14)
annotated-types      ------------------------------     0 B/13.32 KiB
⠙ Preparing packages... (0/14)
annotated-types      ------------------------------     0 B/13.32 KiB
⠙ Preparing packages... (0/14)
annotated-types      ------------------------------     0 B/13.32 KiB
typing-inspection    ------------------------------     0 B/14.21 KiB
⠙ Preparing packages... (0/14)
annotated-types      ------------------------------     0 B/13.32 KiB
typing-inspection    ------------------------------     0 B/14.21 KiB
⠙ Preparing packages... (0/14)
annotated-types      ------------------------------ 13.32 KiB/13.32 KiB
typing-inspection    ------------------------------     0 B/14.21 

In [ ]:
! uv add anthropic python-dotenv nest_asyncio

Resolved 18 packages in 124ms
⠙ Preparing packages... (0/3)
⠙ Preparing packages... (0/3)
⠙ Preparing packages... (0/3)
⠙ Preparing packages... (0/3)
nest-asyncio         ------------------------------     0 B/5.07 KiB
⠙ Preparing packages... (0/3)
nest-asyncio         ------------------------------ 5.07 KiB/5.07 KiB
⠙ Preparing packages... (0/3)
nest-asyncio         ------------------------------ 5.07 KiB/5.07 KiB
⠙ Preparing packages... (0/3)
nest-asyncio         ------------------------------ 5.07 KiB/5.07 KiB
⠙ Preparing packages... (0/3)
nest-asyncio         ------------------------------ 5.07 KiB/5.07 KiB
python-dotenv        ------------------------------ 14.91 KiB/19.78 KiB
⠙ Preparing packages... (0/3)
python-dotenv        ------------------------------ 14.91 KiB/19.78 KiB
⠙ Preparing packages... (0/3)
python-dotenv        ------------------------------ 19.78 KiB/19.78 KiB
⠙ Preparing packages... (0/3)
⠙ Preparing packages... (0/3)
⠙ Preparing packages... (0/3)
⠙ Preparing pac

In [ ]:
! uv add mcp

Resolved 27 packages in 249ms
⠙ Preparing packages... (0/8)
⠙ Preparing packages... (0/8)
⠙ Preparing packages... (0/8)
sse-starlette        ------------------------------     0 B/10.36 KiB
⠙ Preparing packages... (0/8)
sse-starlette        ------------------------------ 10.36 KiB/10.36 KiB
⠙ Preparing packages... (0/8)
sse-starlette        ------------------------------ 10.36 KiB/10.36 KiB
⠙ Preparing packages... (0/8)
sse-starlette        ------------------------------ 10.36 KiB/10.36 KiB
⠙ Preparing packages... (0/8)
sse-starlette        ------------------------------ 10.36 KiB/10.36 KiB
pydantic-settings    ------------------------------     0 B/43.32 KiB
⠙ Preparing packages... (0/8)
sse-starlette        ------------------------------ 10.36 KiB/10.36 KiB
pydantic-settings    ------------------------------     0 B/43.32 KiB
⠙ Preparing packages... (0/8)
sse-starlette        ------------------------------ 10.36 KiB/10.36 KiB
pydantic-settings    ------------------------------     0 

In [ ]:
# Do this in a terminal not in a jupyter notebook
! uv run mcp_chatbot.py